In [55]:
import pandas as pd
import numpy as np
import scipy.signal

In [5]:
data = pd.read_csv('data/gip_A00054817001.csv')
data.tail()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.183466,0.183467,0.183468,0.183469,0.183470,0.183471,0.183472,0.183473,0.183474,0.183475
105,-0.10341,-0.087197,-0.10509,-0.41212,-0.57702,-0.8714,-0.97448,-0.74531,-0.742440,-0.995110,...,2.6328,2.1227,2.7132,2.7088,2.4890,2.2535,2.2961,2.7385,2.8674,2.8515
106,32.53700,32.712000,32.65700,31.95700,32.01600,31.4300,31.23300,31.01700,30.408000,30.923000,...,9.8256,11.2490,11.0690,10.8830,10.4910,10.1880,10.1060,9.6142,9.0291,8.9832
107,17.61500,18.274000,18.85900,18.39900,18.49800,17.9560,18.09500,18.38600,18.681000,18.968000,...,9.4699,10.0530,10.8040,9.4442,8.6215,8.1817,8.3117,8.1053,7.2559,7.2372
108,-2.54660,-2.138500,-1.60930,-1.43180,-1.45440,-1.8480,-1.38760,-0.80250,-0.020022,-0.023821,...,5.3179,5.2205,6.3262,5.5829,4.5769,3.9396,4.0191,4.2779,3.7108,3.4992
109,-16.19000,-16.290000,-16.09700,-15.91000,-16.10600,-15.8610,-15.81600,-15.56700,-15.269000,-15.631000,...,-4.3834,-5.2000,-4.7576,-4.6186,-4.7283,-4.8413,-4.7195,-4.2925,-3.9459,-3.9014


In [6]:
events = pd.read_csv('data/gip_A00054817001_events.csv')
events

,type,value,latency,duration,urevent,Unnamed: 5
0,90,trigger,2091,0,1,NaN
1,type,value,latency,duration,urevent,NaN
2,20,trigger,31225,0,2,NaN
3,type,value,latency,duration,urevent,NaN
4,30,trigger,41224,0,3,NaN
5,type,value,latency,duration,urevent,NaN
6,20,trigger,61223,0,4,NaN
7,type,value,latency,duration,urevent,NaN
8,30,trigger,71223,0,5,NaN
9,type,value,latency,duration,urevent,NaN


In [18]:
# The first 20 mark is at latency 31225
data.columns[31225]

'0.31225'

In [46]:
events['type'] = events['type'].astype('string')
event_lt = list(events['latency'].loc[events['type'] == '20  '].astype('int'))

event_lt

[31225, 61223, 91222, 121222, 151221, 181225]

In [50]:
eyes_open = []

for latency in event_lt:
    try:
        chunk = data.loc[: ,data.columns[latency]:data.columns[latency + 10000]]
    except:
        chunk = data.loc[: ,data.columns[latency]:]
    eyes_open.append(chunk)

,0.61223,0.61224,0.61225,0.61226,0.61227,0.61228,0.61229,0.61230,0.61231,0.61232,...,0.71214,0.71215,0.71216,0.71217,0.71218,0.71219,0.71220,0.71221,0.71222,0.71223
0,3.13290,5.56840,5.090500,4.53130,4.13190,2.83920,3.943900,4.33760,5.57760,7.22540,...,-3.22910,-3.03890,-3.68380,-3.04980,-4.98920,-4.75580,-4.095000,-3.432400,-3.722400,-5.41320
1,1.92990,3.67180,3.425200,3.02270,2.89930,2.11490,2.261000,3.81570,5.40980,5.82810,...,-4.57590,-5.19570,-5.77800,-4.79870,-6.07300,-5.68870,-5.468700,-3.932000,-4.571500,-6.75060
2,0.23601,3.76700,2.486800,2.09740,2.59930,1.47110,2.132600,3.55680,5.13210,6.25950,...,-4.33050,-3.78810,-3.80540,-3.39670,-5.75470,-5.58020,-4.881900,-3.779200,-4.373100,-6.30580
3,-1.75800,0.80653,0.089763,-0.24701,0.15100,-0.42371,-0.048157,0.56913,1.80180,2.34490,...,-2.59170,-2.38450,-2.02550,-2.03340,-4.00920,-3.75330,-3.241400,-2.608500,-3.203200,-5.17030
4,-3.39680,-1.95540,-3.064700,-2.88520,-2.61060,-3.13160,-2.984600,-2.45380,-0.88286,-0.44789,...,-0.80724,-0.89655,-0.48288,-0.55211,-1.63820,-1.93540,-1.503900,-1.318000,-2.016100,-3.55540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,1.87230,1.27450,3.194100,2.37160,1.15130,1.22170,1.651000,0.89602,0.95270,1.37520,...,-1.05520,-0.91758,-1.79210,-1.80100,-2.33340,-2.21330,-1.888000,-1.736700,-2.122100,-2.21800
106,3.88080,0.95414,4.099300,3.99670,2.63500,2.92170,1.629100,1.50540,1.84420,1.10470,...,-3.49560,-4.44180,-5.59880,-5.91880,-4.94000,-4.25880,-4.530800,-3.495800,-3.709700,-3.82650
107,3.48250,2.20500,4.259300,4.36250,3.29390,2.92670,2.041000,3.23620,3.88720,3.21920,...,-5.17380,-6.74090,-7.40120,-6.67840,-6.21660,-5.41880,-5.805200,-4.854600,-5.029400,-6.14930
108,2.22340,2.30690,3.244300,3.15320,2.91220,2.77250,1.979900,3.43620,5.04790,4.49510,...,-5.80590,-7.36090,-8.11730,-7.16030,-7.34990,-6.64250,-6.731400,-5.004700,-5.405900,-7.40550


In [79]:
i = 1
chunk_list = []

# Ask George about the equivalent parameter for 2 sec window next meeting
window = ??

# Going through each chunk in eyes open
for chunk in eyes_open:
    # Selecting each channel from the chunk
    for channel in range(len(chunk)):
        sub_ch = chunk.iloc[channel, :].to_numpy(dtype='float')
        # Feeding the single channel/row of the chunk through pwelch function to return f, pxx
        f, pxx = scipy.signal.welch(sub_ch, 500, window='', noverlap=.5*len(sub_ch))
        chunk_list.append((i, f, pxx))
    i += 1


chunk_list    

[(1,
  array([0.        , 0.00390625, 0.0078125 , 0.01171875, 0.015625  ,
         0.01953125, 0.0234375 , 0.02734375, 0.03125   , 0.03515625,
         0.0390625 , 0.04296875, 0.046875  , 0.05078125, 0.0546875 ,
         0.05859375, 0.0625    , 0.06640625, 0.0703125 , 0.07421875,
         0.078125  , 0.08203125, 0.0859375 , 0.08984375, 0.09375   ,
         0.09765625, 0.1015625 , 0.10546875, 0.109375  , 0.11328125,
         0.1171875 , 0.12109375, 0.125     , 0.12890625, 0.1328125 ,
         0.13671875, 0.140625  , 0.14453125, 0.1484375 , 0.15234375,
         0.15625   , 0.16015625, 0.1640625 , 0.16796875, 0.171875  ,
         0.17578125, 0.1796875 , 0.18359375, 0.1875    , 0.19140625,
         0.1953125 , 0.19921875, 0.203125  , 0.20703125, 0.2109375 ,
         0.21484375, 0.21875   , 0.22265625, 0.2265625 , 0.23046875,
         0.234375  , 0.23828125, 0.2421875 , 0.24609375, 0.25      ,
         0.25390625, 0.2578125 , 0.26171875, 0.265625  , 0.26953125,
         0.2734375 , 0.277343

In [98]:
chunk_df = pd.DataFrame(chunk_list, columns=['chunk', 'f', 'pxx'])
chunk_df.loc[chunk_df['chunk'] == 1]


,chunk,f,pxx
0,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[146.68575046512075, 715.5594168009445, 463.31..."
1,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[233.1373499381473, 1125.5490435864458, 871.69..."
2,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[337.64316858023557, 1660.0783430785293, 1235...."
3,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[273.9100595028295, 1328.8565102948346, 993.12..."
4,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[161.74291261933067, 822.856690981479, 608.614..."
...,...,...,...
105,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[15.566123474650238, 102.1149332211929, 45.460..."
106,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[63.28489652518057, 525.5097356216548, 132.809..."
107,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[70.39629542176803, 531.113722233379, 235.0263..."
108,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[146.72929601744048, 814.5191176904061, 581.36..."


In [100]:
chunk_df.loc[chunk_df['chunk'] == 2]

,chunk,f,pxx
110,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[203.10067134328133, 1416.941353483076, 702.52..."
111,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[249.12722047838744, 1753.9437315598177, 953.1..."
112,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[457.4440598448577, 3249.584538391735, 1562.93..."
113,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[346.85556145927575, 2453.9953387822443, 1155...."
114,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[239.85310180670228, 1591.8530493007604, 703.7..."
...,...,...,...
215,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[11.322423545272382, 70.93537869909851, 41.786..."
216,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[144.255062339989, 854.1757372700736, 262.5867..."
217,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[103.14163228580806, 537.2795421989218, 185.97..."
218,2,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[151.68682617034924, 917.5009981985474, 509.15..."


In [109]:
average_pxx = []
for channel in range(len(chunk_df.loc[chunk_df['chunk'] == 1])):
    for j in range(len(chunk_df['pxx'])):
        chunk1 = chunk_df.loc[chunk_df['chunk'] == 1][channel]['pxx'][j]
        chunk2 = chunk_df.loc[chunk_df['chunk'] == 2]['pxx'][j]
        chunk3 = chunk_df.loc[chunk_df['chunk'] == 3]['pxx'][j]
        chunk4 = chunk_df.loc[chunk_df['chunk'] == 4]['pxx'][j]
        chunk5 = chunk_df.loc[chunk_df['chunk'] == 5]['pxx'][j]
        chunk6 = chunk_df.loc[chunk_df['chunk'] == 6]['pxx'][j]

        total = chunk1 + chunk2 + chunk3 + chunk4 + chunk5 + chunk6
        average = total / 6
        average_pxx.append(average)

KeyError: 0

In [113]:
chunk_df.loc[chunk_df['chunk'] == 1]

,chunk,f,pxx
0,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[146.68575046512075, 715.5594168009445, 463.31..."
1,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[233.1373499381473, 1125.5490435864458, 871.69..."
2,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[337.64316858023557, 1660.0783430785293, 1235...."
3,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[273.9100595028295, 1328.8565102948346, 993.12..."
4,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[161.74291261933067, 822.856690981479, 608.614..."
...,...,...,...
105,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[15.566123474650238, 102.1149332211929, 45.460..."
106,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[63.28489652518057, 525.5097356216548, 132.809..."
107,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[70.39629542176803, 531.113722233379, 235.0263..."
108,1,"[0.0, 0.00390625, 0.0078125, 0.01171875, 0.015...","[146.72929601744048, 814.5191176904061, 581.36..."
